In [ ]:
import argparse
import cv2
import numpy as np
import os

In [ ]:
parser = argparse.ArgumentParser(description="Camera Intrinsic Calibration")
parser.add_argument('-input', '--INPUT_TYPE', default='camera', type=str, help='Input Source: camera/video/image')
parser.add_argument('-type', '--CAMERA_TYPE', default='fisheye', type=str, help='Camera Type: fisheye/normal')
parser.add_argument('-id', '--CAMERA_ID', default=1, type=int, help='Camera ID')
parser.add_argument('-path', '--INPUT_PATH', default='./data/', type=str, help='Input Video/Image Path')
parser.add_argument('-video', '--VIDEO_FILE', default='video.mp4', type=str, help='Input Video File Name (eg.: video.mp4)')
parser.add_argument('-image', '--IMAGE_FILE', default='img_raw', type=str, help='Input Image File Name Prefix (eg.: img_raw)')
parser.add_argument('-mode', '--SELECT_MODE', default='auto', type=str, help='Image Select Mode: auto/manual')
parser.add_argument('-fw','--FRAME_WIDTH', default=1280, type=int, help='Camera Frame Width')
parser.add_argument('-fh','--FRAME_HEIGHT', default=1024, type=int, help='Camera Frame Height')
parser.add_argument('-bw','--BORAD_WIDTH', default=7, type=int, help='Chess Board Width (corners number)')
parser.add_argument('-bh','--BORAD_HEIGHT', default=6, type=int, help='Chess Board Height (corners number)')
parser.add_argument('-size','--SQUARE_SIZE', default=10, type=int, help='Chess Board Square Size (mm)')
parser.add_argument('-num','--CALIB_NUMBER', default=5, type=int, help='Least Required Calibration Frame Number')
parser.add_argument('-delay','--FRAME_DELAY', default=12, type=int, help='Capture Image Time Interval (frame number)')
parser.add_argument('-subpix','--SUBPIX_REGION', default=5, type=int, help='Corners Subpix Optimization Region')
parser.add_argument('-fps','--CAMERA_FPS', default=20, type=int, help='Camera Frame per Second(FPS)')
parser.add_argument('-fs', '--FOCAL_SCALE', default=0.5, type=float, help='Camera Undistort Focal Scale')
parser.add_argument('-ss', '--SIZE_SCALE', default=1, type=float, help='Camera Undistort Size Scale')
parser.add_argument('-store','--STORE_FLAG', default=False, type=bool, help='Store Captured Images (Ture/False)')
parser.add_argument('-store_path', '--STORE_PATH', default='./data/', type=str, help='Path to Store Captured Images')
parser.add_argument('-crop','--CROP_FLAG', default=False, type=bool, help='Crop Input Video/Image to (fw,fh) (Ture/False)')
parser.add_argument('-resize','--RESIZE_FLAG', default=False, type=bool, help='Resize Input Video/Image to (fw,fh) (Ture/False)')
args = parser.parse_args([])                 # Jupyter Notebook에서는 직접 실행할 때 []를 추가해야 하고, py 파일에서는 제거해야 합니다.

In [ ]:
# args.INPUT_TYPE = 'image'                  # 입력 양식 카메라/비디오/이미지
# args.CAMERA_TYPE = 'normal'                # 카메라 유형 어안/일반
# args.CAMERA_ID = 1                         # 카메라 번호
# args.INPUT_PATH = './data/'                # 사진 및 비디오 입력 경로
# args.VIDEO_FILE = 'video.mp4'              # 입력 동영상 파일 이름(확장자 포함)
# args.IMAGE_FILE = 'raw'                    # 이미지 파일명 접두사 입력
# args.SELECT_MODE = 'manual'                # 자동/수동 모드 선택
# args.FRAME_WIDTH = 1280                    # 카메라 해상도 프레임 폭
# args.FRAME_HEIGHT = 720                    # 카메라 해상도 프레임 높이
# args.BORAD_WIDTH = 7                       # 보드 너비 [안쪽 모서리 포인트]
# args.BORAD_HEIGHT = 6                      # 보드의 높이 [안쪽 모서리 점]
# args.SQUARE_SIZE = 10                      # 보드 사각형의 길이(체스판 격자) mm
# args.CALIB_NUMBER = 10                     # 보정된 사진 샘플의 최소 개수를 초기화
# args.FRAME_DELAY = 15                      # 샘플 사이의 프레임 수
# args.SUBPIX_REGION = 3                     # 코너 좌표의 서브 픽셀 최적화를 위한 검색 영역 크기(이미지 해상도에 맞게 조정)
# args.STORE_FLAG = True                     # 캡처한 이미지 저장 여부
# args.STORE_PATH = './data/'                # 캡처한 이미지를 저장할 경로
# args.CROP_FLAG = True                      # 입력 비디오/이미지 크기를 프레임 너비 및 프레임 높이의 설정 값으로 자를지 여부
# args.RESIZE_FLAG = True                    # 입력 비디오/이미지 크기를 프레임 너비 및 프레임 높이 설정에 맞게 조정할지 여부(이미지 크기를 조정하면 카메라 초점이 변경됨)

In [ ]:
# 매개변수 수정을 위한 외부 호출
def getInCalibArgs():
    return args

def editInCalibArgs(new_args):
    global args
    args = new_args

In [ ]:
class CalibData:                             # 보정 데이터 클래스
    def __init__(self):
        self.type = None                     # 사용자 지정 데이터 유형
        self.camera_mat = None               # 카메라 인사이더
        self.dist_coeff = None               # 왜곡 파라미터
        self.rvecs = None                    # 회전 벡터
        self.tvecs = None                    # 번역 벡터
        self.map1 = None                     # 매핑 매트릭스 1
        self.map2 = None                     # 매핑 매트릭스 2
        self.reproj_err = None               # 재투영 오류
        self.ok = False                      # 데이터 수집 완료 플래그

In [ ]:
# cv2.fisheye.calibrate ( objectPoints,      # 보드의 모서리 점의 공간 좌표 벡터
#                         imagePoints,       # 이미지의 모서리 점 좌표 벡터
#                         image_size,        # 이미지 크기
#                         K,                 # 카메라 내부 레퍼런스 매트릭스
#                         D,                 # 수차 파라미터 벡터
#                         rvecs,             # 회전 벡터
#                         tvecs,             # 번역 벡터
#                         flags,             # 작업 플래그
#                         criteria           # 반복 최적화 알고리즘의 중지 기준
#                         )

# flags:
#     cv2.fisheye.CALIB_USE_INTRINSIC_GUESS     # 카메라 내부 레퍼런스 행렬에 fx, fy, cx, cy에 대한 유효한 초기값이 포함되어 있으면 이 값은 더욱 최적화됩니다.
#                                               # 그렇지 않으면 (cx, cy)가 처음에 이미지 중심으로 설정되고(imageSize 사용) 초점 거리가 최소 제곱으로 계산됩니다.
#     cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC     # 외부 파라미터는 각 내부 파라미터 최적화 반복 후에 다시 계산됩니다.
#     cv2.fisheye.CALIB_CHECK_COND              # 조건 번호의 유효성 확인
#     cv2.fisheye.CALIB_FIX_SKEW                # 왜곡 계수(알파)가 0으로 설정되고 0으로 유지됩니다.
#     cv2.fisheye.CALIB_FIX_K1 (K1-K4)          # 선택된 왜곡 계수가 0으로 설정되고 0으로 유지됨 CALIB_FIX_INTRINSIC은 모두 0입니다.

# criteria:
#     TermCriteria (int type, int maxCount, double epsilon)      # 유형, 최대 카운트 수, 최소 정확도
#     Criteria type, can be one of: COUNT, EPS or COUNT + EPS    # 코너 최적화를 위한 최대 반복 횟수 또는 코너 최적화 위치 이동이 엡실론 값보다 작습니다.

In [ ]:
# cv2.checkRange 널이 아닌 요소와 이상값이 없는지 확인하기

In [ ]:
class Fisheye:           # 어안 카메라
    def __init__(self):
        self.data = CalibData()
        self.inited = False
        self.BOARD = np.array([ [(j * args.SQUARE_SIZE, i * args.SQUARE_SIZE, 0.)]
                               for i in range(args.BORAD_HEIGHT) 
                               for j in range(args.BORAD_WIDTH) ],dtype=np.float32)     # 보드 모서리 지점의 2D 좌표(치수를 곱한 값)
        
    # 초기화 및 미세 조정으로 구분되는 캘리브레이션 데이터 업데이트
    def update(self, corners, frame_size):
        board = [self.BOARD] * len(corners)
        if not self.inited:
            self._update_init(board, corners, frame_size)
            self.inited = True
        else:
            self._update_refine(board, corners, frame_size)
        self._calc_reproj_err(corners)
        self._get_undistort_maps()
    
    # 특정 수의 캘리브레이션 샘플을 얻은 경우 초기 캘리브레이션
    def _update_init(self, board, corners, frame_size):
        data = self.data
        data.type = "FISHEYE"
        data.camera_mat = np.eye(3, 3)
        data.dist_coeff = np.zeros((4, 1))
        data.ok, data.camera_mat, data.dist_coeff, data.rvecs, data.tvecs = cv2.fisheye.calibrate(
            board, corners, frame_size, data.camera_mat, data.dist_coeff,
            flags=cv2.fisheye.CALIB_FIX_SKEW|cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC,
            criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 30, 1e-6)) 
        data.ok = data.ok and cv2.checkRange(data.camera_mat) and cv2.checkRange(data.dist_coeff)

    # 미세 조정을 위해 CALIB_USE_INTRINSIC_GUESS를 활성화합니다.
    def _update_refine(self, board, corners, frame_size):
        data = self.data
        data.ok, data.camera_mat, data.dist_coeff, data.rvecs, data.tvecs = cv2.fisheye.calibrate(
            board, corners, frame_size, data.camera_mat, data.dist_coeff,
            flags=cv2.fisheye.CALIB_FIX_SKEW|cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC|cv2.CALIB_USE_INTRINSIC_GUESS,
            criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 10, 1e-6))
        data.ok = data.ok and cv2.checkRange(data.camera_mat) and cv2.checkRange(data.dist_coeff)

    # 픽셀 단위로 재투영 오류 계산하기
    def _calc_reproj_err(self, corners):
        if not self.inited: return
        data = self.data
        data.reproj_err = []
        for i in range(len(corners)):
            corners_reproj, _ = cv2.fisheye.projectPoints(self.BOARD, data.rvecs[i], data.tvecs[i], data.camera_mat, data.dist_coeff)
            err = cv2.norm(corners_reproj, corners[i], cv2.NORM_L2) / len(corners_reproj)
            data.reproj_err.append(err)
            
    # 초점 거리 및 프레임 크기 변경 가능성과 함께 왜곡 제거를 위한 새로운 카메라 내 매개변수 계산
    def _get_camera_mat_dst(self, camera_mat):
        camera_mat_dst = camera_mat.copy()
        camera_mat_dst[0][0] *= args.FOCAL_SCALE
        camera_mat_dst[1][1] *= args.FOCAL_SCALE
        camera_mat_dst[0][2] = args.FRAME_WIDTH / 2 * args.SIZE_SCALE
        camera_mat_dst[1][2] = args.FRAME_HEIGHT / 2 * args.SIZE_SCALE
        return camera_mat_dst
    
    # 왜곡 제거된 매핑 행렬을 계산합니다.
    def _get_undistort_maps(self):
        data = self.data
        camera_mat_dst = self._get_camera_mat_dst(data.camera_mat)
        data.map1, data.map2 = cv2.fisheye.initUndistortRectifyMap(
                                 data.camera_mat, data.dist_coeff, np.eye(3, 3), camera_mat_dst, 
                                 (int(args.FRAME_WIDTH * args.SIZE_SCALE), int(args.FRAME_HEIGHT * args.SIZE_SCALE)), cv2.CV_16SC2)

In [ ]:
# cv2.calibrateCamera (objectPoints,         # 보드의 모서리 점의 공간 좌표 벡터
#                      imagePoints,          # 이미지의 모서리 점 좌표 벡터
#                      image_size,           # 이미지 크기
#                      K,                    # 카메라 내부 레퍼런스 매트릭스
#                      D,                    # 수차 파라미터 벡터
#                      rvecs,                # 회전 벡터
#                      tvecs,                # 번역 벡터
#                      flags,                # 작업 플래그
#                      criteria              # 반복 최적화 알고리즘의 중지 기준
#                     )

# flags:
#     cv2.CALIB_USE_INTRINSIC_GUESS          # 카메라 내부 레퍼런스 행렬에 fx, fy, cx, cy에 대한 유효한 초기값이 포함되어 있으면 이 값은 더욱 최적화됩니다.
#                                            # 그렇지 않으면 (cx, cy)가 처음에 이미지의 중앙으로 설정되고(imageSize 사용) 초점 거리가 최소 제곱으로 계산됩니다.
#     cv2.CALIB_FIX_PRINCIPAL_POINT          # 고정 광축 포인트(CALIB_USE_INTRINSIC_GUESS 설정 시 사용 가능)
#     cv2.CALIB_FIX_ASPECT_RATIO             # fx/fy 값을 고정하는 이 함수는 fy만 자유 파라미터로 간주합니다.
#     cv2.CALIB_ZERO_TANGENT_DIST            # 접선 왜곡 계수(p1, p2)를 0으로 설정하고 0으로 유지합니다.
#     cv2.CALIB_FIX_FOCAL_LENGTH             # CALIB_USE_INTRINSIC_GUESS를 설정하면 글로벌 최적화 중에 초점 거리가 변경되지 않습니다.
#     cv2.CALIB_FIX_K1 (K1-K6)               # 해당 방사형 왜곡 계수를 0 또는 지정된 초기 값으로 수정합니다.
#     cv2.CALIB_RATIONAL_MODEL               # 이상적인 모델: 계수 k4, k5 및 k6 활성화. 현재 8개 이상의 계수가 반환됩니다.
#     cv2.CALIB_THIN_PRISM_MODEL             # 씬 프리즘 모델: 계수 s1, s2, s3 및 s4를 활성화합니다. 현재 12개 이상의 계수가 반환됩니다.
#     cv2.CALIB_FIX_S1_S2_S3_S4              # 씬 프리즘 왜곡 계수를 0 또는 지정된 초기 값으로 수정했습니다.
#     cv2.CALIB_TILTED_MODEL                 # 기울기 모델: 계수 tauX 및 tauY를 활성화합니다. 이 시점에서 14개의 계수가 반환됩니다.
#     cv2.CALIB_FIX_TAUX_TAUY                # 계수가 0 또는 지정된 초기 값으로 고정된 기울기 센서 모델

In [ ]:
class Normal:           # 평면 카메라
    def __init__(self):
        self.data = CalibData()
        self.inited = False
        self.BOARD = np.array([ [(j * args.SQUARE_SIZE, i * args.SQUARE_SIZE, 0.)]
                               for i in range(args.BORAD_HEIGHT) 
                               for j in range(args.BORAD_WIDTH) ],dtype=np.float32)
        
    def update(self, corners, frame_size):
        board = [self.BOARD] * len(corners)
        if not self.inited:
            self._update_init(board, corners, frame_size)
            self.inited = True
        else:
            self._update_refine(board, corners, frame_size)
        self._calc_reproj_err(corners)
        self._get_undistort_maps()
        
    def _update_init(self, board, corners, frame_size):
        data = self.data
        data.type = "NORMAL"
        data.camera_mat = np.eye(3, 3)
        data.dist_coeff = np.zeros((5, 1))     # 왜곡 벡터의 크기는 사용된 모델에 따라 수정됩니다.
        data.ok, data.camera_mat, data.dist_coeff, data.rvecs, data.tvecs = cv2.calibrateCamera(
            board, corners, frame_size, data.camera_mat, data.dist_coeff, 
            criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 30, 1e-6))
        data.ok = data.ok and cv2.checkRange(data.camera_mat) and cv2.checkRange(data.dist_coeff)
        
    def _update_refine(self, board, corners, frame_size):
        data = self.data
        data.ok, data.camera_mat, data.dist_coeff, data.rvecs, data.tvecs = cv2.calibrateCamera(
            board, corners, frame_size, data.camera_mat, data.dist_coeff,  
            flags = cv2.CALIB_USE_INTRINSIC_GUESS,
            criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 10, 1e-6))
        data.ok = data.ok and cv2.checkRange(data.camera_mat) and cv2.checkRange(data.dist_coeff)
        
    def _calc_reproj_err(self, corners):
        if not self.inited: return
        data = self.data
        data.reproj_err = []
        for i in range(len(corners)):
            corners_reproj, _ = cv2.projectPoints(self.BOARD, data.rvecs[i], data.tvecs[i], data.camera_mat, data.dist_coeff)
            err = cv2.norm(corners_reproj, corners[i], cv2.NORM_L2) / len(corners_reproj)
            data.reproj_err.append(err)
            
    def _get_camera_mat_dst(self, camera_mat):
        camera_mat_dst = camera_mat.copy()
        camera_mat_dst[0][0] *= args.FOCAL_SCALE
        camera_mat_dst[1][1] *= args.FOCAL_SCALE
        camera_mat_dst[0][2] = args.FRAME_WIDTH / 2 * args.SIZE_SCALE
        camera_mat_dst[1][2] = args.FRAME_HEIGHT / 2 * args.SIZE_SCALE
        return camera_mat_dst
    
    def _get_undistort_maps(self):
        data = self.data
        camera_mat_dst = self._get_camera_mat_dst(data.camera_mat)
        data.map1, data.map2 = cv2.initUndistortRectifyMap(
                                 data.camera_mat, data.dist_coeff, np.eye(3, 3), camera_mat_dst, 
                                 (int(args.FRAME_WIDTH * args.SIZE_SCALE), int(args.FRAME_HEIGHT * args.SIZE_SCALE)), cv2.CV_16SC2)

In [ ]:
# cv2.findChessboardCorners ( image,         # 보드 이미지
#                             patternSize,   # 보드 그리드의 행과 열에 있는 [내부 모서리 점] 개수
#                             corners,       # 출력 배열
#                             flags          # 작업 플래그
#                             )
# flags:
#     CV_CALIB_CB_ADAPTIVE_THRESH            # 적응형 임계값을 사용하여 이미지를 흑백으로 변환하기
#     CV_CALIB_CB_NORMALIZE_IMAGE            # 이미지를 정규화합니다.
#     CV_CALIB_CB_FILTER_QUADS               # 윤곽 검색 단계에서 추출된 잘못된 사각형을 필터링합니다.
#     CALIB_CB_FAST_CHECK                    # 이미지를 빠르게 확인하여 체스판의 모서리를 찾습니다.

In [ ]:
# cv2.cornerSubPix (image,                        # 보드 이미지
#                   corners,                      # 바둑판 모서리 점(수학.)
#                   winSize,                      # 검색 창 길이의 절반
#                   zeroZone,                     # 검색 영역의 데드 존 크기의 절반인 (-1,-1)은 검색 영역이 없음을 나타냅니다.
#                   criteria                      # 반복 중지 기준
#                  )

In [ ]:
# cv2.fisheye.initUndistortRectifyMap (K,         # 카메라 내부 레퍼런스 매트릭스
#                                      D,         # 왜곡 벡터
#                                      R,         # 회전 매트릭스
#                                      P,         # 새로운 카메라 매트릭스
#                                      size,      # 출력 이미지 크기
#                                      m1type,    # 매핑 매트릭스 유형
#                                      map1,      # 출력 매핑 행렬 1
#                                      map2       # 출력 매핑 매트릭스 2
#                                     )

In [ ]:
class InCalibrator:                  # 내부 참조 캘리브레이터
    def __init__(self, camera):
        if camera == 'fisheye':
            self.camera = Fisheye()  # 어안 카메라 클래스
        elif camera == 'normal':
            self.camera = Normal()   # 일반 카메라 클래스
        else:
            raise Exception("camera should be fisheye/normal")
        self.corners = []
    
    # 매개변수 수정을 위한 외부 호출에 대한 인수 매개변수 가져오기
    @staticmethod
    def get_args():
        return args
    
    # 보드 그리드의 모서리 점의 좌표 가져오기
    def get_corners(self, img):
        ok, corners = cv2.findChessboardCorners(img, (args.BORAD_WIDTH, args.BORAD_HEIGHT),
                      flags = cv2.CALIB_CB_ADAPTIVE_THRESH|cv2.CALIB_CB_NORMALIZE_IMAGE|cv2.CALIB_CB_FAST_CHECK)
        if ok: 
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # 코너 좌표의 서브 픽셀 최적화
            corners = cv2.cornerSubPix(gray, corners, (args.SUBPIX_REGION, args.SUBPIX_REGION), (-1, -1),
                                       (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01))
        return ok, corners
    
    # 다이어그램에 바둑판 격자의 모서리 그리기
    def draw_corners(self, img):
        ok, corners = self.get_corners(img)
        cv2.drawChessboardCorners(img, (args.BORAD_WIDTH, args.BORAD_HEIGHT), corners, ok)
        return img
    
    # 이미지 왜곡 제거
    def undistort(self, img):
        data = self.camera.data
        return cv2.remap(img, data.map1, data.map2, cv2.INTER_LINEAR)
    
    # 기존 코너 포인트 좌표를 사용한 보정
    def calibrate(self, img):
        if len(self.corners) >= args.CALIB_NUMBER:
            self.camera.update(self.corners, img.shape[1::-1])  # 캘리브레이션 데이터 업데이트
        return self.camera.data
    
    def __call__(self, raw_frame):
        ok, corners = self.get_corners(raw_frame)
        result = self.camera.data
        if ok:
            self.corners.append(corners)          # 새 코너 좌표 추가
            result = self.calibrate(raw_frame)    # 보정 결과 가져오기
        return result

In [ ]:
# 중앙 자르기
def centerCrop(img,width,height):
    if img.shape[1] < width or img.shape[0] < height:
        raise Exception("CROP size should be smaller than original size")
    img = img[round((img.shape[0]-height)/2) : round((img.shape[0]-height)/2)+height,
              round((img.shape[1]-width)/2) : round((img.shape[1]-width)/2)+width ]  
    return img

In [ ]:
# 이미지 파일 필터링
def get_images(PATH, NAME):
    filePath = [os.path.join(PATH, x) for x in os.listdir(PATH) 
                if any(x.endswith(extension) for extension in ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG'])
               ]                                                            # 지정된 경로에서 모든 이미지 파일 가져오기
    filenames = [filename for filename in filePath if NAME in filename]     # 그런 다음 지정된 이름이 포함된 이미지를 필터링합니다.
    if len(filenames) == 0:
        raise Exception("from {} read images failed".format(PATH))
    return filenames

In [ ]:
class CalibMode():
    def __init__(self, calibrator, input_type, mode):
        self.calibrator = calibrator
        self.input_type = input_type
        self.mode = mode
    
    # 이미지 사전 처리
    def imgPreprocess(self, img):
        if args.CROP_FLAG:          # 이미지 크기 자르기
            img = centerCrop(img, args.FRAME_WIDTH, args.FRAME_HEIGHT)
        elif args.RESIZE_FLAG:      # 이미지 크기 조정
            img = cv2.resize(img, (args.FRAME_WIDTH, args.FRAME_HEIGHT))
        return img
    
    # 카메라 설정하기
    def setCamera(self, cap):
        cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M','J','P','G'))    # 인코딩 형식을 MJPG로 설정
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, args.FRAME_WIDTH)                      # 카메라 해상도 설정
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, args.FRAME_HEIGHT)
        cap.set(cv2.CAP_PROP_FPS, args.CAMERA_FPS)                               # 카메라 프레임 속도 설정
        return cap
    
    # 캘리브레이션 프로그램 실행
    def runCalib(self, raw_frame, display_raw=True, display_undist=True):
        calibrator = self.calibrator
        raw_frame = self.imgPreprocess(raw_frame)
        result = calibrator(raw_frame)
        raw_frame = calibrator.draw_corners(raw_frame)
        if display_raw:
            cv2.namedWindow("raw_frame", flags = cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
            cv2.imshow("raw_frame", raw_frame)
        if len(calibrator.corners) > args.CALIB_NUMBER and display_undist: 
            undist_frame = calibrator.undistort(raw_frame)
            cv2.namedWindow("undist_frame", flags = cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
            cv2.imshow("undist_frame", undist_frame)   
        cv2.waitKey(1)
        return result
    
    # 영상 입력 자동 보정
    def imageAutoMode(self):
        calibrator = self.calibrator
        filenames = get_images(args.INPUT_PATH, args.IMAGE_FILE)
        for filename in filenames:
            print(filename)
            raw_frame = cv2.imread(filename)
            result = self.runCalib(raw_frame)
            key = cv2.waitKey(1)
            if key == 27: break
        cv2.destroyAllWindows() 
        return result
    
    # 사진 입력 수동 선택 스페이스바를 눌러 확인 기타 키를 눌러 사진을 삭제합니다.
    def imageManualMode(self):
        filenames = get_images(args.INPUT_PATH, args.IMAGE_FILE)
        for filename in filenames:
            print(filename)
            raw_frame = cv2.imread(filename)
            raw_frame = self.imgPreprocess(raw_frame)
            img = raw_frame.copy()
            img = self.calibrator.draw_corners(img)
            display = "raw_frame: press SPACE to SELECT, other key to SKIP, press ESC to QUIT"
            cv2.namedWindow(display, flags = cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
            cv2.imshow(display, img)
            key = cv2.waitKey(0)
            if key == 32:
                result = self.runCalib(raw_frame, display_raw = False)
            if key == 27: break
        cv2.destroyAllWindows() 
        return result
    
    # 비디오 입력 자동 보정
    def videoAutoMode(self):
        cap = cv2.VideoCapture(args.INPUT_PATH + args.VIDEO_FILE)
        if not cap.isOpened(): 
            raise Exception("from {} read video failed".format(args.INPUT_PATH + args.VIDEO_FILE))
        frame_id = 0
        while True:
            key = cv2.waitKey(1)
            ok, raw_frame = cap.read()
            raw_frame = self.imgPreprocess(raw_frame)
            if frame_id % args.FRAME_DELAY == 0:
                if args.STORE_FLAG:  # 프레임 저장
                    cv2.imwrite(args.STORE_PATH + 'img_raw{}.jpg'.format(len(self.calibrator.corners)), raw_frame)
                result = self.runCalib(raw_frame) 
                print(len(self.calibrator.corners))
            frame_id += 1 
            key = cv2.waitKey(1)
            if key == 27: break
        cap.release()
        cv2.destroyAllWindows() 
        return result
    
    # 비디오 입력 수동 선택 스페이스바를 눌러 사진을 캡처합니다.
    def videoManualMode(self):
        cap = cv2.VideoCapture(args.INPUT_PATH + args.VIDEO_FILE)
        if not cap.isOpened(): 
            raise Exception("from {} read video failed".format(args.INPUT_PATH + args.VIDEO_FILE))
        while True:
            key = cv2.waitKey(1)
            ok, raw_frame = cap.read()
            raw_frame = self.imgPreprocess(raw_frame)
            display = "raw_frame: press SPACE to capture image"
            cv2.namedWindow(display, flags = cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
            cv2.imshow(display, raw_frame)
            if key == 32:
                if args.STORE_FLAG:  # 프레임 저장
                    cv2.imwrite(args.STORE_PATH + 'img_raw{}.jpg'.format(len(self.calibrator.corners)), raw_frame)
                result = self.runCalib(raw_frame) 
                print(len(self.calibrator.corners))
            if key == 27: break
        cap.release()
        cv2.destroyAllWindows() 
        return result
    
    # 카메라 입력 온라인 보정
    def cameraAutoMode(self):
        cap = cv2.VideoCapture(args.CAMERA_ID)
        if not cap.isOpened(): 
            raise Exception("from {} read video failed".format(args.CAMERA_ID))
        cap = self.setCamera(cap)
        frame_id = 0
        start_flag = False
        while True:
            key = cv2.waitKey(1)
            ok, raw_frame = cap.read()
            raw_frame = self.imgPreprocess(raw_frame)
            if key == 32: start_flag = True
            if key == 27: break
            if not start_flag:
                cv2.putText(raw_frame, 'press SPACE to start!', (args.FRAME_WIDTH//4,args.FRAME_HEIGHT//2), 
                             cv2.FONT_HERSHEY_COMPLEX, 1.5, (0,0,255), 2)
                cv2.imshow("raw_frame", raw_frame)
                continue
            if frame_id % args.FRAME_DELAY == 0:
                if args.STORE_FLAG:  # 프레임 저장
                    cv2.imwrite(args.STORE_PATH + 'img_raw{}.jpg'.format(len(self.calibrator.corners)), raw_frame)
                result = self.runCalib(raw_frame) 
                print(len(self.calibrator.corners))
            frame_id += 1 
        cap.release()
        cv2.destroyAllWindows() 
        return result
    
    # 카메라 입력 수동 선택 스페이스바를 눌러 사진을 캡처합니다.
    def cameraManualMode(self):
        cap = cv2.VideoCapture(args.CAMERA_ID)
        if not cap.isOpened(): 
            raise Exception("from {} read video failed".format(args.CAMERA_ID))
        cap = self.setCamera(cap)
        frame_id = 0
        while True:
            key = cv2.waitKey(1)
            ok, raw_frame = cap.read()
            raw_frame = self.imgPreprocess(raw_frame)
            display = "raw_frame: press SPACE to capture image"
            cv2.namedWindow(display, flags = cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
            cv2.imshow(display, raw_frame)
            if key == 32:
                if args.STORE_FLAG:  # 프레임 저장
                    cv2.imwrite(args.STORE_PATH + 'img_raw{}.jpg'.format(len(self.calibrator.corners)), raw_frame)
                result = self.runCalib(raw_frame) 
                print(len(self.calibrator.corners))
            if key == 27: break
        cap.release()
        cv2.destroyAllWindows() 
        return result

    def __call__(self):
        input_type = self.input_type
        mode = self.mode
        if input_type == 'image' and mode == 'auto':
            result = self.imageAutoMode()
        if input_type == 'image' and mode == 'manual':
            result = self.imageManualMode()
        if input_type == 'video' and mode == 'auto':
            result = self.videoAutoMode()
        if input_type == 'video' and mode == 'manual':
            result = self.videoManualMode()
        if input_type == 'camera' and mode == 'auto':
            result = self.cameraAutoMode()
        if input_type == 'camera' and mode == 'manual':
            result = self.cameraManualMode()
        return result

In [ ]:
def main():
    calibrator = InCalibrator(args.CAMERA_TYPE)                            # 내부 기준 캘리브레이터 초기화
    calib = CalibMode(calibrator, args.INPUT_TYPE, args.SELECT_MODE)       # 보정 모드 선택
    result = calib()                                                       # 보정 시작
                  
    if len(calibrator.corners) == 0:                      # 캘리브레이션 실패 보드를 찾을 수 없음 또는 매개변수 설정 오류
        raise Exception("Calibration failed. Chessboard not found, check the parameters")  
    if len(calibrator.corners) < args.CALIB_NUMBER:       # 캘리브레이션 샘플이 초기 캘리브레이션에 필요한 이미지 수보다 적습니다.
        raise Exception("Warning: Calibration images are not enough. At least {} valid images are needed.".format(args.CALIB_NUMBER))            

    print("Calibration Complete")
    print("Camera Matrix is : {}".format(result.camera_mat.tolist()))                 # 카메라 인사이더
    print("Distortion Coefficient is : {}".format(result.dist_coeff.tolist()))        # 왜곡 벡터
    print("Reprojection Error is : {}".format(np.mean(result.reproj_err)))            # 평균 재투사 오류
    np.save('camera_{}_K.npy'.format(args.CAMERA_ID),result.camera_mat.tolist())
    np.save('camera_{}_D.npy'.format(args.CAMERA_ID),result.dist_coeff.tolist())      # 데이터 출력 및 저장
        
if __name__ == '__main__':
    main()
